In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# CSV-Datei laden (angenommene Struktur)
df = pd.read_csv('dataset/clf_cat/eye_movements.csv')  # Passe den Pfad an deine Datei an

# Merkmale und Zieltrennung
X = df.iloc[:, :-1]  # Alle Spalten außer der letzten
y = df.iloc[:, -1]   # Die letzte Spalte

# KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)  # KNN mit k=5

# Experiment 10 Mal durchführen
n_experiments = 10
build_times = np.zeros(n_experiments)
evaluate_times = np.zeros(n_experiments)
accuracies = np.zeros(n_experiments)

for seed in range(n_experiments):
    print(f"Experiment {seed + 1} mit Seed {seed}")
    
    # Daten aufteilen (66% Training, 34% Test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=seed)
    
    # Zeitmessung für das Modelltraining
    start_build = time.time()
    knn.fit(X_train, y_train)
    end_build = time.time()
    build_times[seed] = end_build - start_build
    
    # Zeitmessung für die Modellbewertung
    start_evaluate = time.time()
    y_pred = knn.predict(X_test)
    end_evaluate = time.time()
    evaluate_times[seed] = end_evaluate - start_evaluate
    
    # Genauigkeit berechnen
    accuracies[seed] = accuracy_score(y_test, y_pred)

# Ergebnisse als Arrays ausgeben
print("Array der Trainingszeiten (build model):", build_times)
print("Array der Bewertungszeiten (evaluate):", evaluate_times)
print("Array der Genauigkeiten:", accuracies)

# Durchschnittsergebnisse
print(f"\nDurchschnittliche Trainingszeit (build model): {np.mean(build_times):.4f} Sekunden")
print(f"Durchschnittliche Bewertungszeit (evaluate): {np.mean(evaluate_times):.4f} Sekunden")
print(f"Durchschnittliche Genauigkeit: {np.mean(accuracies):.2%}")


Experiment 1 mit Seed 0
Experiment 2 mit Seed 1
Experiment 3 mit Seed 2
Experiment 4 mit Seed 3
Experiment 5 mit Seed 4
Experiment 6 mit Seed 5
Experiment 7 mit Seed 6
Experiment 8 mit Seed 7
Experiment 9 mit Seed 8
Experiment 10 mit Seed 9
Array der Trainingszeiten (build model): [0.00299621 0.00200105 0.0030005  0.00399756 0.00300097 0.00300002
 0.00199699 0.0030005  0.00200129 0.00300002]
Array der Bewertungszeiten (evaluate): [0.17831731 0.1400044  0.11099887 0.13601494 0.10998273 0.15901518
 0.11598349 0.15501475 0.11098242 0.13800049]
Array der Genauigkeiten: [0.56745265 0.55353692 0.56049478 0.56474681 0.55392346 0.55856204
 0.55276382 0.53807499 0.55508311 0.55005798]

Durchschnittliche Trainingszeit (build model): 0.0028 Sekunden
Durchschnittliche Bewertungszeit (evaluate): 0.1354 Sekunden
Durchschnittliche Genauigkeit: 55.55%


In [2]:
import pandas as pd
import time
import weka.core.jvm as jvm
from weka.classifiers import Classifier, Evaluation
from weka.core.converters import Loader
from weka.core.classes import Random
from weka.filters import Filter
import numpy as np

# JVM starten
jvm.start()

# CSV-Datei laden und in WEKA-Format umwandeln
loader = Loader(classname="weka.core.converters.CSVLoader")
data = loader.load_file('dataset/clf_cat/eye_movements.csv')  # Passe den Pfad an deine Datei an
data.class_is_last()  # Die Zielspalte (letzte Spalte) als Klassenattribut setzen

# Filter anwenden, falls Zielvariable numerisch ist und zu nominal konvertiert werden muss
if data.class_attribute.is_numeric:
    num_to_nominal = Filter(classname="weka.filters.unsupervised.attribute.NumericToNominal", options=["-R", "last"])
    num_to_nominal.inputformat(data)
    data = num_to_nominal.filter(data)

# KNN Classifier in WEKA
knn = Classifier(classname="weka.classifiers.lazy.IBk", options=["-K", "5"])  # KNN mit k=5

# Experiment 10 Mal durchführen
n_experiments = 10
build_times = np.zeros(n_experiments)
evaluate_times = np.zeros(n_experiments)
accuracies = np.zeros(n_experiments)

for seed in range(n_experiments):
    print(f"Experiment {seed + 1} mit Seed {seed}")

    # Daten zufällig aufteilen (66% Training, 34% Test)
    train, test = data.train_test_split(66, Random(seed))

    # Zeitmessung für das Modelltraining
    start_build = time.time()
    knn.build_classifier(train)
    end_build = time.time()
    build_times[seed] = end_build - start_build

    # Zeitmessung für die Modellbewertung
    evaluator = Evaluation(train)
    start_evaluate = time.time()
    evaluator.test_model(knn, test)
    end_evaluate = time.time()
    evaluate_times[seed] = end_evaluate - start_evaluate

    # Genauigkeit berechnen
    accuracy = evaluator.percent_correct / 100
    accuracies[seed] = accuracy

# JVM stoppen
jvm.stop()

# Ergebnisse als Arrays ausgeben
print("Array der Trainingszeiten (build model):", build_times)
print("Array der Bewertungszeiten (evaluate):", evaluate_times)
print("Array der Genauigkeiten:", accuracies)

# Durchschnittsergebnisse
print(f"\nDurchschnittliche Trainingszeit (build model): {np.mean(build_times):.4f} Sekunden")
print(f"Durchschnittliche Bewertungszeit (evaluate): {np.mean(evaluate_times):.4f} Sekunden")
print(f"Durchschnittliche Genauigkeit: {np.mean(accuracies):.2%}")


DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['c:\\Users\\jakob\\anaconda3\\Lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'c:\\Users\\jakob\\anaconda3\\Lib\\site-packages\\weka\\lib\\core.jar', 'c:\\Users\\jakob\\anaconda3\\Lib\\site-packages\\weka\\lib\\mtj.jar', 'c:\\Users\\jakob\\anaconda3\\Lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\jakob\\anaconda3\\Lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


Experiment 1 mit Seed 0
Experiment 2 mit Seed 1
Experiment 3 mit Seed 2
Experiment 4 mit Seed 3
Experiment 5 mit Seed 4
Experiment 6 mit Seed 5
Experiment 7 mit Seed 6
Experiment 8 mit Seed 7
Experiment 9 mit Seed 8
Experiment 10 mit Seed 9
Array der Trainingszeiten (build model): [0.01220584 0.00189829 0.0029974  0.06299829 0.00199866 0.00195885
 0.00200009 0.00199604 0.00100112 0.00209332]
Array der Bewertungszeiten (evaluate): [0.91033936 0.84051657 0.85721016 0.7825923  0.84136033 0.86428952
 0.89442158 0.93312693 0.87348723 0.92778635]
Array der Genauigkeiten: [0.57595671 0.56938539 0.57866254 0.58214148 0.58175493 0.56474681
 0.56861229 0.5775029  0.57093158 0.57170468]

Durchschnittliche Trainingszeit (build model): 0.0091 Sekunden
Durchschnittliche Bewertungszeit (evaluate): 0.8725 Sekunden
Durchschnittliche Genauigkeit: 57.41%
